In [ ]:
%load_ext autoreload
%autoreload 2
import pickle
import IPython.display as ipd
import iisignature as iisig
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import cluster
import pyximport
pyximport.install()
%load_ext Cython
import sigkernel as ksig
from utils.data import *

# Data preprocessing

In [ ]:
key = 'single_key'
sample_len = 30

In [ ]:
with open(f'./data/dataframes/{key}/df_titles_30_min_notes_pitch_range_5_24.pkl', 'rb') as f:
    df_titles = pickle.load(f)
len(df_titles)

### Transform to gap, duration, delta pitch format

In [ ]:
gap_dur_dpitch_dfs = gap_duration_deltapitch_transform([item[0] for item in df_titles])
dataset = GapDurationDeltaPitchDataset(gap_dur_dpitch_dfs, sample_len=sample_len, stride=10000)
len(dataset) == len(df_titles)

### Transform to rectilinear path

In [ ]:
X = []
for i in range(len(dataset)):
    X.append(dataset[i])
X = torch.stack(X)
X = batch_rectilinear_with_gap_transform(X)
Xs = X.numpy()
Xs.shape, X.shape

### Calculate signature and cluster

In [ ]:
signatures = []
for path in Xs:
    signatures.append(iisig.sig(path, 5))
signatures = np.array(signatures)
signatures.shape

In [ ]:
labels = cluster.KMeans(n_clusters=4).fit(signatures).labels_
plt.hist(labels)

### Compute Gram matrix of signature kernel with rational quadratic static kernel

In [ ]:
static_kernel = ksig.static.kernels.RationalQuadraticKernel(sigma=1.0)
kernel = ksig.kernels.SignatureKernel(n_levels=5, order=1, normalization=0, static_kernel=static_kernel, device_ids=None)

In [ ]:
# calculate gram matrix in batches
# batch_size = 15
# gram_matrix = torch.empty(len(X), len(X))
# for i in range(int(len(X) / batch_size)+1):
#     print((i+1)*batch_size)
#     gram_matrix[i*batch_size:(i+1)*batch_size] = kernel(X[i*batch_size:(i+1)*batch_size].to('cuda'), X.to('cuda')).cpu()
# torch.save(gram_matrix, f'./data/gram_matrices/gram_matrix_{sample_len}_min_notes_pitch_range_5_24.pt')

In [ ]:
gram_matrix = torch.load(f'./data/gram_matrices/gram_matrix_{sample_len}_min_notes_pitch_range_5_24.pt')
gram_matrix = torch.tril(gram_matrix) + torch.tril(gram_matrix, diagonal=-1).T # make it symmetric as numerical errors make it not symmetric
gram_matrix.shape

In [ ]:
sns.heatmap(torch.tril(gram_matrix, diagonal=-1), cmap='viridis')

### Analyse gram matrix and cluster to eliminate outliers

In [ ]:
# Test alignment of kernel matrix with list of dataframe indices
start = 79
print([item[1:] for item in df_titles[start:start+1]])
sample_idx = 79
midi_data = df_to_midi(batch_gap_duration_pitch_to_df(dataset[sample_idx].unsqueeze(0))[0])
fs = 44100
audio_data = midi_data.fluidsynth(fs=fs)
ipd.Audio(audio_data, rate=fs)

In [ ]:
# get coordinates (x,y) of highest values in gram matrix
n = 10
max_coords_value = []
temp_matrix = torch.tril(gram_matrix, diagonal=-1)
for i in range(n):
    c = torch.argmax(temp_matrix).item()
    max_coords_value.append(((c//len(X), c%len(X)), torch.max(temp_matrix).item()))
    temp_matrix[c//len(X), c%len(X)] = 0
max_coords_value

In [ ]:
fig, ax = plt.subplots(n//5, 5, figsize=(20, 4*n//5))
for i in range(len(max_coords_value)):
    ax[i//5, i%5].plot(X[max_coords_value[i][0][0]].numpy()[:,0], X[max_coords_value[i][0][0]].numpy()[:,1])
    ax[i//5, i%5].plot(X[max_coords_value[i][0][1]].numpy()[:,0], X[max_coords_value[i][0][1]].numpy()[:,1])

In [ ]:
# get coordinates (x,y) of lowest values in gram matrix
n = 10
min_coords_value = []
temp_matrix = torch.tril(gram_matrix, diagonal=-1)
for i in range(n):
    c = torch.argmin(temp_matrix).item()
    min_coords_value.append(((c//len(X), c%len(X)), torch.min(temp_matrix).item()))
    temp_matrix[c//len(X), c%len(X)] = np.inf
min_coords_value

In [ ]:
fig, ax = plt.subplots(n//5, 5, figsize=(20, 4*n//5))
for i in range(len(min_coords_value)):
    ax[i//5, i%5].plot(X[min_coords_value[i][0][0]].numpy()[:,0], X[min_coords_value[i][0][0]].numpy()[:,1])
    ax[i//5, i%5].plot(X[min_coords_value[i][0][1]].numpy()[:,0], X[min_coords_value[i][0][1]].numpy()[:,1])

In [ ]:
sample_idx = min_coords_value[6][0][0]
print(sample_idx)
print(df_titles[sample_idx][1:])
midi_data = df_to_midi(df_titles[sample_idx][0])
fs = 44100
audio_data = midi_data.fluidsynth(fs=fs)
ipd.Audio(audio_data, rate=fs)

### Spectral clustering

In [ ]:
n_clusters = 5
labels = cluster.SpectralClustering(n_clusters=n_clusters, affinity='precomputed').fit(gram_matrix).labels_
plt.hist(labels, bins=n_clusters)

In [ ]:
print(len(labels) == len(df_titles))
df_titles_labels = [(df_titles[i][0], df_titles[i][1], df_titles[i][2], df_titles[i][3], labels[i]) for i in range(len(df_titles))]
with open(f'./data/dataframes/{key}/df_titles_cluster_30_min_notes_pitch_range_5_24.pkl', 'wb') as f:
    pickle.dump(df_titles_labels, f)

In [ ]:
df_titles_labels[0]

In [ ]:
counts = np.unique(labels, return_counts=True)
counts

In [ ]:
# get args of label 4
args = [i for i in range(len(labels)) if labels[i] == 5]
args

In [ ]:
audio_datas = []
for arg in args:
    print(df_titles[arg][1:])
    midi_data = df_to_midi(df_titles[arg][0])
    fs = 44100
    audio_datas.append(midi_data.fluidsynth(fs=fs))

In [ ]:
ipd.Audio(audio_datas[3], rate=fs)

In [ ]:
df_titles = [item for i, item in enumerate(df_titles) if i not in args]
with open(f'./data/dataframes/single_key/df_titles_30_min_notes_pitch_range_5_24_spec_10_v1.pkl', 'wb') as f:
    pickle.dump(df_titles, f)
len(df_titles)

In [ ]:
# sort from largest to smallest then remove row and column in order
print(gram_matrix.shape)
for arg in np.sort(args)[::-1]:
    gram_matrix = torch.cat((gram_matrix[:arg], gram_matrix[arg+1:]))
    gram_matrix = torch.cat((gram_matrix[:,:arg], gram_matrix[:,arg+1:]), dim=1)
print(gram_matrix.shape)

### Affinity propagation clustering

In [ ]:
labels = cluster.AffinityPropagation(affinity='precomputed').fit(gram_matrix).labels_
counts = plt.hist(labels, bins=len(np.unique(labels)))

### Agglomerative clustering

In [ ]:
# Convert gram matrix to distance matrix using Cauchy-Schwarz
# d_CS(x,y) = arccos k^2(x,y)/k(x,x)k(y,y).

# dist_matrix = torch.tril(gram_matrix, diagonal=-1)
# for i in range(len(X)):
#     dist_matrix[i:,i] /= torch.sqrt(gram_matrix[i,i]) * torch.diag(gram_matrix).sqrt()[i:]
# dist_matrix = torch.acos(dist_matrix)
# dist_matrix = torch.tril(dist_matrix, diagonal=-1) + torch.tril(dist_matrix, diagonal=-1).T
# dist_matrix

In [ ]:
# Convert gram matrix to distance matrix using |u-v|^2 = k(x,x) + k(y,y) - 2k(x,y)

dist_matrix = -2*torch.tril(gram_matrix, diagonal=-1)
for i in range(len(X)):
    dist_matrix[i:,i] += gram_matrix[i,i] + torch.diag(gram_matrix)[i:]
dist_matrix = torch.sqrt(dist_matrix)
dist_matrix = torch.tril(dist_matrix, diagonal=-1) + torch.tril(dist_matrix, diagonal=-1).T
dist_matrix

In [ ]:
sns.heatmap(dist_matrix, cmap='viridis')

In [ ]:
n_clusters = 3
labels = cluster.AgglomerativeClustering(n_clusters=n_clusters, metric='precomputed', linkage='average').fit(dist_matrix).labels_
plt.hist(labels, bins=n_clusters)

### DBScan clustering

In [ ]:
eps = np.mean(dist_matrix[torch.tril(dist_matrix, diagonal=-1) > 0].numpy())
labels = cluster.DBSCAN(eps=eps, metric='precomputed').fit(dist_matrix).labels_
plt.hist(labels, bins=len(np.unique(labels)))